In [1]:
using Random, DataFrames, CSV
import Pkg; Pkg.add("StatsBase")
include("src/world.jl"); include("src/value.jl"); include("src/policy.jl"); include("src/sim.jl")

gw = World.GridWorld(5,5, Set([(3,3)])) # a wall at (3,3)
goals = [(5,5), (5,1), (1,5)]
start = (1,1)

# Precompute V,Q per goal
Qs = Dict{Tuple{Int,Int}, Function}()
for g in goals
    Vg = Value.compute_state_values(gw, g)
    Qdict = Value.compute_Q(gw, Vg, g)
    # convenience: closure Q(s,a)
    Qs[g] = (s,a)->Qdict[(s[1],s[2],a)]
end

# Generate episodes
rng = MersenneTwister(7)
rows = DataFrame(episode=Int[], gx=Int[], gy=Int[], mind=String[], param=Float64[],
                 actions=Vector{Int}[], states=Vector{Tuple{Int,Int}}[])
eid = 0
for g in goals
    for β in [0.5, 1.0, 2.0, 6.0]
        for _ in 1:50
            eid+=1
            ep = Sim.simulate_episode(gw; start, goal=g, mind=:softmax_β, param=β, Q_sa=Qs[g], rng)
            push!(rows, (eid, g[1], g[2], "softmax_β", β, Int.(ep.actions), ep.states))
        end
    end
end

CSV.write("episodes.csv", rows)

   Resolving package versions...
  No Changes to `~/Desktop/Yale_Fall_2025/Algorithms_of_Mind/final_project/final_project_code/Project.toml`
  No Changes to `~/Desktop/Yale_Fall_2025/Algorithms_of_Mind/final_project/final_project_code/Manifest.toml`


"episodes.csv"

In [2]:
include("src/observer.jl")
betas = [0.5, 1.0, 2.0, 6.0]
ep = first(eachrow(rows)) # or pick any episode
g_true = (ep.gx, ep.gy)
post = Observer.posterior_grid(gw, goals, betas, Qs, ep.states, World.Action.(ep.actions))
println("Posterior over (goal, β):")
for ((gx,gy,β), p) in sort(collect(post), by=last, rev=true)
    println("G=($gx,$gy), β=$β => $(round(p,digits=3))")
end

Posterior over (goal, β):
G=(5,5), β=0.5 => 0.484
G=(5,1), β=0.5 => 0.309
G=(1,5), β=0.5 => 0.206
G=(5,5), β=1.0 => 0.001
G=(5,1), β=1.0 => 0.0
G=(1,5), β=1.0 => 0.0
G=(5,5), β=2.0 => 0.0
G=(5,1), β=2.0 => 0.0
G=(1,5), β=2.0 => 0.0
G=(5,5), β=6.0 => 0.0
G=(5,1), β=6.0 => 0.0
G=(1,5), β=6.0 => 0.0
